In [ ]:
# --------------------------------------------------------------------
## Import Libraries
# --------------------------------------------------------------------

import numpy as np
from pathlib import Path
import sys
import configparser
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm

from scipy.integrate import solve_ivp
from scipy.optimize import least_squares
from scipy.interpolate import interp1d

from joblib import Parallel, delayed
from tqdm import tqdm


# -----------------------------
## File Paths
# -----------------------------

# Current Directory
current_dir = Path.cwd()
# Top Directory
top_dir = current_dir.parent if current_dir.name == '5-Bootstrap' else current_dir

# Libs Directory
libs_dir = str(top_dir / "0-Libs")
# Config
config_dir = top_dir / "0-Config"

# Data Directory
data_dir = top_dir / "0-Data"   
    # HighRes (0)
highres_dir = str(data_dir / "0-HighRes")
    # Routine (1)
routine_dir = str(data_dir / "1-Routine")
    # Active (2)
active_dir = str(data_dir / "2-Active")
    # LongActive (3)
long_active_dir = str(data_dir / "3-LongActive")
    # LongRoutine (4)
long_routine_dir = str(data_dir / "4-LongRoutine")

# Sensitivity Results Directory
sensitivity_results_dir = top_dir / "2-Sensitivity" / "Results"

# PCA Results File
pca_results_path = str(top_dir / "4-Correlations" / "Results" / "PCA_Results.pkl")

# Results Directory
results_dir = (top_dir / "5-Bootstrap" / "Results")

# -----------------------------
## Import Libraries - Custom
# -----------------------------
sys.path.append(libs_dir)

from plant_config import get_reactor_initial_values
from asm3_model import ode_system_wrapper
from asm3_model_sunode import ode_system_sunode

# -----------------------------
## Configs
# -----------------------------

# Config File
config = configparser.ConfigParser()
config.read(config_dir / "config.ini")
   # Seed for random number generator
seed = int(config['OVERALL']['seed'])
np.random.seed(seed)                         # Set random seed
data_to_use_for_sampling = str(config['OVERALL']['data_to_use_for_run'])
    # Reactor volumes
r1_V = float(config['REACTOR']['r1_V'])        # Volume of reactor 1
    # Parameter Ranges
range_param_k_H = tuple(map(float, config['PARAMRANGES']['range_param_k_H'].split(',')))              # Range of k_H
range_param_K_X = tuple(map(float, config['PARAMRANGES']['range_param_K_X'].split(',')))              # Range of K_X
range_param_k_STO = tuple(map(float, config['PARAMRANGES']['range_param_small_k_STO'].split(',')))          # Range of k_STO
range_param_eta_NOX = tuple(map(float, config['PARAMRANGES']['range_param_eta_NOX'].split(',')))      # Range of eta_NOX
range_param_K_O2 = tuple(map(float, config['PARAMRANGES']['range_param_K_O2'].split(',')))            # Range of K_O2
range_param_K_NOX = tuple(map(float, config['PARAMRANGES']['range_param_K_NOX'].split(',')))          # Range of K_NOX
range_param_K_S = tuple(map(float, config['PARAMRANGES']['range_param_K_S'].split(',')))              # Range of K_S
range_param_K_STO = tuple(map(float, config['PARAMRANGES']['range_param_big_K_STO'].split(',')))          # Range of K_STO
range_param_mu_H = tuple(map(float, config['PARAMRANGES']['range_param_mu_H'].split(',')))            # Range of mu_H
range_param_K_NH4 = tuple(map(float, config['PARAMRANGES']['range_param_K_NH4'].split(',')))          # Range of K_NH4
range_param_K_ALK = tuple(map(float, config['PARAMRANGES']['range_param_K_ALK'].split(',')))          # Range of K_ALK
range_param_b_H_O2 = tuple(map(float, config['PARAMRANGES']['range_param_b_H_O2'].split(',')))        # Range of b_H_O2
range_param_b_H_NOX = tuple(map(float, config['PARAMRANGES']['range_param_b_H_NOX'].split(',')))      # Range of b_H_NOX
range_param_b_STO_O2 = tuple(map(float, config['PARAMRANGES']['range_param_b_STO_O2'].split(',')))    # Range of b_STO_O2
range_param_b_STO_NOX = tuple(map(float, config['PARAMRANGES']['range_param_b_STO_NOX'].split(',')))  # Range of b_STO_NOX
range_param_mu_A = tuple(map(float, config['PARAMRANGES']['range_param_mu_A'].split(',')))            # Range of mu_A
range_param_K_A_NH4 = tuple(map(float, config['PARAMRANGES']['range_param_K_A_NH4'].split(',')))      # Range of K_A_NH4
range_param_K_A_O2 = tuple(map(float, config['PARAMRANGES']['range_param_K_A_O2'].split(',')))        # Range of K_A_O2
range_param_K_A_ALK = tuple(map(float, config['PARAMRANGES']['range_param_K_A_ALK'].split(',')))      # Range of K_A_ALK
range_param_b_A_O2 = tuple(map(float, config['PARAMRANGES']['range_param_b_A_O2'].split(',')))        # Range of b_A_O2
range_param_b_A_NOX = tuple(map(float, config['PARAMRANGES']['range_param_b_A_NOX'].split(',')))      # Range of b_A_NOX
range_param_f_S_I = tuple(map(float, config['PARAMRANGES']['range_param_f_S_I'].split(',')))          # Range of f_S_I
range_param_Y_STO_O2 = tuple(map(float, config['PARAMRANGES']['range_param_Y_STO_O2'].split(',')))    # Range of Y_STO_O2
range_param_Y_STO_NOX = tuple(map(float, config['PARAMRANGES']['range_param_Y_STO_NOX'].split(',')))  # Range of Y_STO_NOX
range_param_Y_H_O2 = tuple(map(float, config['PARAMRANGES']['range_param_Y_H_O2'].split(',')))        # Range of Y_H_O2
range_param_Y_H_NOX = tuple(map(float, config['PARAMRANGES']['range_param_Y_H_NOX'].split(',')))      # Range of Y_H_NOX
range_param_Y_A = tuple(map(float, config['PARAMRANGES']['range_param_Y_A'].split(',')))              # Range of Y_A
range_param_f_X_I = tuple(map(float, config['PARAMRANGES']['range_param_f_X_I'].split(',')))          # Range of f_X_I
range_param_i_N_S_I = tuple(map(float, config['PARAMRANGES']['range_param_i_N_S_I'].split(',')))      # Range of i_N_S_I
range_param_i_N_S_S = tuple(map(float, config['PARAMRANGES']['range_param_i_N_S_S'].split(',')))      # Range of i_N_S_S
range_param_i_N_X_I = tuple(map(float, config['PARAMRANGES']['range_param_i_N_X_I'].split(',')))      # Range of i_N_X_I
range_param_i_N_X_S = tuple(map(float, config['PARAMRANGES']['range_param_i_N_X_S'].split(',')))      # Range of i_N_X_S
range_param_i_N_BM = tuple(map(float, config['PARAMRANGES']['range_param_i_N_BM'].split(',')))        # Range of i_N_BM
range_param_i_SS_X_I = tuple(map(float, config['PARAMRANGES']['range_param_i_SS_X_I'].split(',')))    # Range of i_SS_X_I
range_param_i_SS_X_S = tuple(map(float, config['PARAMRANGES']['range_param_i_SS_X_S'].split(',')))    # Range of i_SS_X_S
range_param_i_SS_BM = tuple(map(float, config['PARAMRANGES']['range_param_i_SS_BM'].split(',')))      # Range of i_SS_BM
    # True values
true_param_k_H = float(config['TRUEPARAMS']['true_param_k_H'])          
true_param_K_X = float(config['TRUEPARAMS']['true_param_K_X'])          
true_param_k_STO = float(config['TRUEPARAMS']['true_param_small_k_STO'])      
true_param_eta_NOX = float(config['TRUEPARAMS']['true_param_eta_NOX'])  
true_param_K_O2 = float(config['TRUEPARAMS']['true_param_K_O2'])
true_param_K_NOX = float(config['TRUEPARAMS']['true_param_K_NOX'])
true_param_K_S = float(config['TRUEPARAMS']['true_param_K_S'])
true_param_K_STO = float(config['TRUEPARAMS']['true_param_big_K_STO'])
true_param_mu_H = float(config['TRUEPARAMS']['true_param_mu_H'])
true_param_K_NH4 = float(config['TRUEPARAMS']['true_param_K_NH4'])
true_param_K_ALK = float(config['TRUEPARAMS']['true_param_K_ALK'])
true_param_b_H_O2 = float(config['TRUEPARAMS']['true_param_b_H_O2'])
true_param_b_H_NOX = float(config['TRUEPARAMS']['true_param_b_H_NOX'])
true_param_b_STO_O2 = float(config['TRUEPARAMS']['true_param_b_STO_O2'])
true_param_b_STO_NOX = float(config['TRUEPARAMS']['true_param_b_STO_NOX'])
true_param_mu_A = float(config['TRUEPARAMS']['true_param_mu_A'])
true_param_K_A_NH4 = float(config['TRUEPARAMS']['true_param_K_A_NH4'])
true_param_K_A_O2 = float(config['TRUEPARAMS']['true_param_K_A_O2'])
true_param_K_A_ALK = float(config['TRUEPARAMS']['true_param_K_A_ALK'])
true_param_b_A_O2 = float(config['TRUEPARAMS']['true_param_b_A_O2'])
true_param_b_A_NOX = float(config['TRUEPARAMS']['true_param_b_A_NOX'])
true_param_f_S_I = float(config['TRUEPARAMS']['true_param_f_S_I'])
true_param_Y_STO_O2 = float(config['TRUEPARAMS']['true_param_Y_STO_O2'])
true_param_Y_STO_NOX = float(config['TRUEPARAMS']['true_param_Y_STO_NOX'])
true_param_Y_H_O2 = float(config['TRUEPARAMS']['true_param_Y_H_O2'])
true_param_Y_H_NOX = float(config['TRUEPARAMS']['true_param_Y_H_NOX'])
true_param_Y_A = float(config['TRUEPARAMS']['true_param_Y_A'])
true_param_f_X_I = float(config['TRUEPARAMS']['true_param_f_X_I'])
true_param_i_N_S_I = float(config['TRUEPARAMS']['true_param_i_N_S_I'])
true_param_i_N_S_S = float(config['TRUEPARAMS']['true_param_i_N_S_S'])
true_param_i_N_X_I = float(config['TRUEPARAMS']['true_param_i_N_X_I'])
true_param_i_N_X_S = float(config['TRUEPARAMS']['true_param_i_N_X_S'])
true_param_i_N_BM = float(config['TRUEPARAMS']['true_param_i_N_BM'])
true_param_i_SS_X_I = float(config['TRUEPARAMS']['true_param_i_SS_X_I'])
true_param_i_SS_X_S = float(config['TRUEPARAMS']['true_param_i_SS_X_S'])
true_param_i_SS_BM = float(config['TRUEPARAMS']['true_param_i_SS_BM'])

true_theta = {
    'k_H': true_param_k_H,
    'K_X': true_param_K_X,
    'k_STO': true_param_k_STO,
    'eta_NOX': true_param_eta_NOX,
    'K_O2': true_param_K_O2,
    'K_NOX': true_param_K_NOX,
    'K_S': true_param_K_S,
    'K_STO': true_param_K_STO,
    'mu_H': true_param_mu_H,
    'K_NH4': true_param_K_NH4,
    'K_ALK': true_param_K_ALK,
    'b_H_O2': true_param_b_H_O2,
    'b_H_NOX': true_param_b_H_NOX,
    'b_STO_O2': true_param_b_STO_O2,
    'b_STO_NOX': true_param_b_STO_NOX,
    'mu_A': true_param_mu_A,
    'K_A_NH4': true_param_K_A_NH4,
    'K_A_O2': true_param_K_A_O2,
    'K_A_ALK': true_param_K_A_ALK,
    'b_A_O2': true_param_b_A_O2,
    'b_A_NOX': true_param_b_A_NOX,
    'f_S_I': true_param_f_S_I,
    'Y_STO_O2': true_param_Y_STO_O2,
    'Y_STO_NOX': true_param_Y_STO_NOX,
    'Y_H_O2': true_param_Y_H_O2,
    'Y_H_NOX': true_param_Y_H_NOX,
    'Y_A': true_param_Y_A,
    'f_X_I': true_param_f_X_I,
    'i_N_S_I': true_param_i_N_S_I,
    'i_N_S_S': true_param_i_N_S_S,
    'i_N_X_I': true_param_i_N_X_I,
    'i_N_X_S': true_param_i_N_X_S,
    'i_N_BM': true_param_i_N_BM,
    'i_SS_X_I': true_param_i_SS_X_I,
    'i_SS_X_S': true_param_i_SS_X_S,
    'i_SS_BM': true_param_i_SS_BM
}

true_theta_array = np.array(list(true_theta.values()))


    # From identifiability
NAASI_threshold = float(config['IDENTIFIABILITY']['NAASI_threshold'])  # NAASI threshold for identifiability
    # Sampling
solver_method = str(config['SAMPLING']['solver_method'])
config_tuning_samples = int(config['SAMPLING']['tuning_samples'])
config_draw_samples = int(config['SAMPLING']['draw_samples'])
config_sample_chains = int(config['SAMPLING']['run_chains'])
config_sample_cores = int(config['SAMPLING']['run_cores'])

# ----------------------------------------------------------
## Load Data from csv
# ----------------------------------------------------------

    # Highres (0)
data_highres_influent_states = pd.read_csv(highres_dir + "/HighRes_Influent_States.csv")
data_highres_effluent_states = pd.read_csv(highres_dir + "/HighRes_Effluent_States.csv")
data_highres_influent_compounds = pd.read_csv(highres_dir + "/HighRes_Influent_Compounds.csv")
data_highres_effluent_compounds = pd.read_csv(highres_dir + "/HighRes_Effluent_Compounds.csv")
    # Routine (1)
data_routine_influent_states = pd.read_csv(routine_dir + "/Routine_Influent_States.csv")
data_routine_effluent_states = pd.read_csv(routine_dir + "/Routine_Effluent_States.csv")
data_routine_influent_compounds = pd.read_csv(routine_dir + "/Routine_Influent_Compounds.csv")
data_routine_effluent_compounds = pd.read_csv(routine_dir + "/Routine_Effluent_Compounds.csv")
    # Active (2)
data_active_influent_states = pd.read_csv(active_dir + "/Active_Influent_States.csv")
data_active_effluent_states = pd.read_csv(active_dir + "/Active_Effluent_States.csv")
data_active_influent_compounds = pd.read_csv(active_dir + "/Active_Influent_Compounds.csv")
data_active_effluent_compounds = pd.read_csv(active_dir + "/Active_Effluent_Compounds.csv")
    # LongActive (3)
data_longactive_influent_states = pd.read_csv(long_active_dir + "/LongActive_Influent_States.csv")
data_longactive_effluent_states = pd.read_csv(long_active_dir + "/LongActive_Effluent_States.csv")
data_longactive_influent_compounds = pd.read_csv(long_active_dir + "/LongActive_Influent_Compounds.csv")
data_longactive_effluent_compounds = pd.read_csv(long_active_dir + "/LongActive_Effluent_Compounds.csv")
    # LongRoutine (4)
data_longroutine_influent_states = pd.read_csv(long_routine_dir + "/LongRoutine_Influent_States.csv")
data_longroutine_effluent_states = pd.read_csv(long_routine_dir + "/LongRoutine_Effluent_States.csv")
data_longroutine_influent_compounds = pd.read_csv(long_routine_dir + "/LongRoutine_Influent_Compounds.csv")
data_longroutine_effluent_compounds = pd.read_csv(long_routine_dir + "/LongRoutine_Effluent_Compounds.csv")

# ----------------------------------------------------------
## Data to use for identifiability
# ----------------------------------------------------------

data_mapping_influent_states = {
    'HighRes': data_highres_influent_states,
    'Routine': data_routine_influent_states,
    'Active': data_active_influent_states,
    'LongActive': data_longactive_influent_states,
    'LongRoutine': data_longroutine_influent_states,
}
data_mapping_effluent_states = {
    'HighRes': data_highres_effluent_states,
    'Routine': data_routine_effluent_states,
    'Active': data_active_effluent_states,
    'LongActive': data_longactive_effluent_states,
    'LongRoutine': data_longroutine_effluent_states,
}
data_mapping_influent_compounds = {
    'HighRes': data_highres_influent_compounds,
    'Routine': data_routine_influent_compounds,
    'Active': data_active_influent_compounds,
    'LongActive': data_longactive_influent_compounds,
    'LongRoutine': data_longroutine_influent_compounds,
}
data_mapping_effluent_compounds = {
    'HighRes': data_highres_effluent_compounds,
    'Routine': data_routine_effluent_compounds,
    'Active': data_active_effluent_compounds,
    'LongActive': data_longactive_effluent_compounds,
    'LongRoutine': data_longroutine_effluent_compounds,
}

try:
    Data_Influent_states = data_mapping_influent_states[data_to_use_for_sampling]
    Data_Effluent_states = data_mapping_effluent_states[data_to_use_for_sampling]
    Data_Influent_compounds = data_mapping_influent_compounds[data_to_use_for_sampling]
    Data_Effluent_compounds = data_mapping_effluent_compounds[data_to_use_for_sampling]
except KeyError:
    raise ValueError("Invalid data for sanpling. Choose from HighRes, Routine, LongRoutine, HalfActive (Deprecated), Active, or LongActive.")

# ----------------------------------------------------------
print("Config and Data loaded")


In [ ]:
# ----------------------------------------------------------
## Parameter Ranges from Priors
# ----------------------------------------------------------

all_param_idx = {
    'k_H': 0,
    'K_X': 1,
    'k_STO': 2,
    'eta_NOX': 3,
    'K_O2': 4,
    'K_NOX': 5,
    'K_S': 6,
    'K_STO': 7,
    'mu_H': 8,
    'K_NH4': 9,
    'K_ALK': 10,
    'b_H_O2': 11,
    'b_H_NOX': 12,
    'b_STO_O2': 13,
    'b_STO_NOX': 14,
    'mu_A': 15,
    'K_A_NH4': 16,
    'K_A_O2': 17,
    'K_A_ALK': 18,
    'b_A_O2': 19,
    'b_A_NOX': 20,
    'f_S_I': 21,
    'Y_STO_O2': 22,
    'Y_STO_NOX': 23,
    'Y_H_O2': 24,
    'Y_H_NOX': 25,
    'Y_A': 26,
    'f_X_I': 27,
    'i_N_S_I': 28,
    'i_N_S_S': 29,
    'i_N_X_I': 30,
    'i_N_X_S': 31,
    'i_N_BM': 32,
    'i_SS_X_I': 33,
    'i_SS_X_S': 34,
    'i_SS_BM': 35
}

# Dictionary with minimum and maximum values for each parameter
theta_ranges = {
    'k_H': range_param_k_H,
    'K_X': range_param_K_X,
    'k_STO': range_param_k_STO,
    'eta_NOX': range_param_eta_NOX,
    'K_O2': range_param_K_O2,
    'K_NOX': range_param_K_NOX,
    'K_S': range_param_K_S,
    'K_STO': range_param_K_STO,
    'mu_H': range_param_mu_H,
    'K_NH4': range_param_K_NH4,
    'K_ALK': range_param_K_ALK,
    'b_H_O2': range_param_b_H_O2,
    'b_H_NOX': range_param_b_H_NOX,
    'b_STO_O2': range_param_b_STO_O2,
    'b_STO_NOX': range_param_b_STO_NOX,
    'mu_A': range_param_mu_A,
    'K_A_NH4': range_param_K_A_NH4,
    'K_A_O2': range_param_K_A_O2,
    'K_A_ALK': range_param_K_A_ALK,
    'b_A_O2': range_param_b_A_O2,
    'b_A_NOX': range_param_b_A_NOX,
    'f_S_I': range_param_f_S_I,
    'Y_STO_O2': range_param_Y_STO_O2,
    'Y_STO_NOX': range_param_Y_STO_NOX,
    'Y_H_O2': range_param_Y_H_O2,
    'Y_H_NOX': range_param_Y_H_NOX,
    'Y_A': range_param_Y_A,
    'f_X_I': range_param_f_X_I,
    'i_N_S_I': range_param_i_N_S_I,
    'i_N_S_S': range_param_i_N_S_S,
    'i_N_X_I': range_param_i_N_X_I,
    'i_N_X_S': range_param_i_N_X_S,
    'i_N_BM': range_param_i_N_BM,
    'i_SS_X_I': range_param_i_SS_X_I,
    'i_SS_X_S': range_param_i_SS_X_S,
    'i_SS_BM': range_param_i_SS_BM,
}

In [ ]:
# ----------------------------------------------------------
## Priors / Means and Standard Deviations of parameters
# ----------------------------------------------------------

# ----------------------------------------------------------
# Priors
    # Normal distribution for all parameters
        # Mean = Midpoint between the range
        # Std = to give range limits as 95% CI -- therefore: Var = (High - Low) / (2 * Critical Value), where Critical Value = 1.96 for 95% CI

alpha = 0.05
crit_value = norm.ppf(1 - (alpha / 2)) # For 95% CI

priors = {
    # Priors for parameters -- mean and std
    'k_H':          [np.mean(range_param_k_H),          (range_param_k_H[1] - range_param_k_H[0]) / (2 * crit_value)],    # (g COD_X_S) / (g COD_X_H * d), Hydrolysis rate constant
    'K_X':          [np.mean(range_param_K_X),          (range_param_K_X[1] - range_param_K_X[0]) / (2 * crit_value)],    # (g COD_X_S) / (g COD_X_H), Hydrolysis saturation constant
    'k_STO':        [np.mean(range_param_k_STO),        (range_param_k_STO[1] - range_param_k_STO[0]) / (2 * crit_value)],    # (g COD_S_S) / (g COD_X_H * d), Storage rate constant
    'eta_NOX':      [np.mean(range_param_eta_NOX),      (range_param_eta_NOX[1] - range_param_eta_NOX[0]) / (2 * crit_value)],    # - , Anoxic reduction factor
    'K_O2':         [np.mean(range_param_K_O2),         (range_param_K_O2[1] - range_param_K_O2[0]) / (2 * crit_value)],    # (g O2 / m3), Saturation constant for S_NO2
    'K_NOX':        [np.mean(range_param_K_NOX),        (range_param_K_NOX[1] - range_param_K_NOX[0]) / (2 * crit_value)],    # (g (NO3-)-N / m3), Saturation constant for S_NOX
    'K_S':          [np.mean(range_param_K_S),          (range_param_K_S[1] - range_param_K_S[0]) / (2 * crit_value)],    # (g COD_S_S / m3), Saturation constant for Substrate S_S
    'K_STO':        [np.mean(range_param_K_STO),        (range_param_K_STO[1] - range_param_K_STO[0]) / (2 * crit_value)],    # (g COD_X_STO / g COD_X_H), Saturation constant for X_STO
    'mu_H':         [np.mean(range_param_mu_H),         (range_param_mu_H[1] - range_param_mu_H[0]) / (2 * crit_value)],    # (d^-1), Heterotrophic maximum specific growth rate of X_H
    'K_NH4':        [np.mean(range_param_K_NH4),        (range_param_K_NH4[1] - range_param_K_NH4[0]) / (2 * crit_value)],    # (g N / m3), Saturation constant for ammonium, S_NH4
    'K_ALK':        [np.mean(range_param_K_ALK),        (range_param_K_ALK[1] - range_param_K_ALK[0]) / (2 * crit_value)],    # (mole HCO3- / m3), Saturation constant for alkalinity for X_H
    'b_H_O2':       [np.mean(range_param_b_H_O2),       (range_param_b_H_O2[1] - range_param_b_H_O2[0]) / (2 * crit_value)],    # (d^-1), Aerobic endogenous respiration rate of X_H
    'b_H_NOX':      [np.mean(range_param_b_H_NOX),      (range_param_b_H_NOX[1] - range_param_b_H_NOX[0]) / (2 * crit_value)],    # (d^-1), Anoxic endogenous respiration rate of X_H
    'b_STO_O2':     [np.mean(range_param_b_STO_O2),     (range_param_b_STO_O2[1] - range_param_b_STO_O2[0]) / (2 * crit_value)],    # (d^-1), Aerobic endogenous respiration rate for X_STO
    'b_STO_NOX':    [np.mean(range_param_b_STO_NOX),    (range_param_b_STO_NOX[1] - range_param_b_STO_NOX[0]) / (2 * crit_value)],    # (d^-1), Anoxic endogenous respiration rate for X_STO
    'mu_A':         [np.mean(range_param_mu_A),         (range_param_mu_A[1] - range_param_mu_A[0]) / (2 * crit_value)],    # (d^-1), Autotrophic maximum specific growth rate of X_A
    'K_A_NH4':      [np.mean(range_param_K_A_NH4),      (range_param_K_A_NH4[1] - range_param_K_A_NH4[0]) / (2 * crit_value)],    # (g N / m3), Ammonium substrate saturation constant for X_A
    'K_A_O2':       [np.mean(range_param_K_A_O2),       (range_param_K_A_O2[1] - range_param_K_A_O2[0]) / (2 * crit_value)],    # (g O2 / m3), Oxygen saturation for nitrifiers
    'K_A_ALK':      [np.mean(range_param_K_A_ALK),      (range_param_K_A_ALK[1] - range_param_K_A_ALK[0]) / (2 * crit_value)],    # (mole HCO3- / m3), Bicarbonate saturation for nitrifiers
    'b_A_O2':       [np.mean(range_param_b_A_O2),       (range_param_b_A_O2[1] - range_param_b_A_O2[0]) / (2 * crit_value)],    # (d^-1), Aerobic endogenous respiration rate of X_A
    'b_A_NOX':      [np.mean(range_param_b_A_NOX),      (range_param_b_A_NOX[1] - range_param_b_A_NOX[0]) / (2 * crit_value)],    # (d^-1), Anoxic endogenous respiration rate of X_A
    'f_S_I':        [np.mean(range_param_f_S_I),        (range_param_f_S_I[1] - range_param_f_S_I[0]) / (2 * crit_value)],    # (g COD_S_I) / (g COD_X_s), Production of S_I in hydrolisis
    'Y_STO_O2':     [np.mean(range_param_Y_STO_O2),     (range_param_Y_STO_O2[1] - range_param_Y_STO_O2[0]) / (2 * crit_value)],    # (g COD_X_STO) / (g COD_S_S), Aerobic yield of stored product per S_S
    'Y_STO_NOX':    [np.mean(range_param_Y_STO_NOX),    (range_param_Y_STO_NOX[1] - range_param_Y_STO_NOX[0]) / (2 * crit_value)],    # (g COD_X_STO) / (g COD_S_S), Anoxic yield of stored product per S_S
    'Y_H_O2':       [np.mean(range_param_Y_H_O2),       (range_param_Y_H_O2[1] - range_param_Y_H_O2[0]) / (2 * crit_value)],    # (g COD_X_H) / (g COD_S_STO), Aerobic yield of heterotrophic biomass
    'Y_H_NOX':      [np.mean(range_param_Y_H_NOX),      (range_param_Y_H_NOX[1] - range_param_Y_H_NOX[0]) / (2 * crit_value)],    # (g COD_X_H) / (g COD_S_STO), Anoxic yield of heterotrophic biomass
    'Y_A':          [np.mean(range_param_Y_A),          (range_param_Y_A[1] - range_param_Y_A[0]) / (2 * crit_value)],    # (g COD_X_A) / (g N_S_NOX), Yield of autotrophic biomass per NO3-N
    'f_X_I':        [np.mean(range_param_f_X_I),        (range_param_f_X_I[1] - range_param_f_X_I[0]) / (2 * crit_value)],    # (g COD_X_I) / (g COD_X_BM), Production of X_I in endogenous repiration
    'i_N_S_I':      [np.mean(range_param_i_N_S_I),      (range_param_i_N_S_I[1] - range_param_i_N_S_I[0]) / (2 * crit_value)],    # (g N) / (g COD_S_I), N content of S_I
    'i_N_S_S':      [np.mean(range_param_i_N_S_S),      (range_param_i_N_S_S[1] - range_param_i_N_S_S[0]) / (2 * crit_value)],    # (g N) / (g COD_S_S), N content of S_S
    'i_N_X_I':      [np.mean(range_param_i_N_X_I),      (range_param_i_N_X_I[1] - range_param_i_N_X_I[0]) / (2 * crit_value)],    # (g N) / (g COD_X_I), N content of X_I
    'i_N_X_S':      [np.mean(range_param_i_N_X_S),      (range_param_i_N_X_S[1] - range_param_i_N_X_S[0]) / (2 * crit_value)],    # (g N) / (g COD_X_S), N content of X_S
    'i_N_BM':       [np.mean(range_param_i_N_BM),       (range_param_i_N_BM[1] - range_param_i_N_BM[0]) / (2 * crit_value)],    # (g N) / (g COD_X_BM), N content of biomass, X_H, X_A
    'i_SS_X_I':     [np.mean(range_param_i_SS_X_I),     (range_param_i_SS_X_I[1] - range_param_i_SS_X_I[0]) / (2 * crit_value)],    # (g SS) / (g COD_X_I), SS to COD ratio for X_I
    'i_SS_X_S':     [np.mean(range_param_i_SS_X_S),     (range_param_i_SS_X_S[1] - range_param_i_SS_X_S[0]) / (2 * crit_value)],    # (g SS) / (g COD_X_S), SS to COD ratio for X_S
    'i_SS_BM':      [np.mean(range_param_i_SS_BM),      (range_param_i_SS_BM[1] - range_param_i_SS_BM[0]) / (2 * crit_value)],    # (g SS) / (g COD_X_BM), SS to COD ratio for biomass, X_H, X_A
}

# ----------------------------------------------------------
print("Priors Defined")

In [ ]:
# -----------------------------------------------------------
## Sensitivity Analysis Results
# -----------------------------------------------------------

NAASI_results = pd.read_csv(sensitivity_results_dir / 'NAASI_combined.csv')
NAASI_results = NAASI_results.set_index('Parameter')
# Get parameters with NAASI values below threshold
params_to_always_fix = {
    param: priors[param][0] for param in NAASI_results.index if NAASI_results.loc[param, 'NAASI'] < NAASI_threshold
}
# Print the parameters to always fix based on NAASI results 
print(f'Parameters to always fix based on NAASI results (Param, value):')
for param, fixed_val in params_to_always_fix.items():
    print(f'    {param}:    {fixed_val}')

always_fixed_param_idxs = [all_param_idx[name] for name in params_to_always_fix.keys()] # Indices of parameters to always fix
# Fix to mean of theta_ranges
always_fixed_param_vals = [(theta_ranges[name][0] + theta_ranges[name][1]) / 2 for name in params_to_always_fix.keys()] # Mean of the parameter ranges



In [ ]:
# -----------------
## PCA Results
# -----------------

# PCA Results from Correlation - PCA previous step
    # Import PCA results from pkl file
pca_results = pd.read_pickle(pca_results_path)
pca_eigenvectors = pca_results['eigenvectors']
pca_scaler_mean = pca_results['scaler_mean']
pca_scaler_scale = pca_results['scaler_scale']
# pca_param_names = ['K_X', 'k_STO', 'K_S', 'mu_H',..] as example
pca_param_names = pca_results['param_names']
pca_n_params = len(pca_param_names)
pca_n_components = pca_results['n_components']

# parameter names specify order in which the parameters are stored in the eigenvectors, therfore what order they will be transformed to
# Print order of parameters to check
print("PCA Parameter Names (Order):")
for i, param in enumerate(pca_param_names):
    print(f"[{i}] {param}")

    # Transpose eigenvectors to get (n_params_pca, n_pca_components)
pca_eigenvectors_transposed = pca_eigenvectors.T

# Indices of PCA parameters in the all_param_idx dictionary
pca_parameter_idxs = [all_param_idx[param] for param in pca_param_names]


In [ ]:
# -----------------------------------------------------------
## Model free, fixed, pca, and zero parameters
# -----------------------------------------------------------


# The following parameters are fixed to prior means:
    # i_SS_BM, i_SS_X_I, i_SS_X_S, f_S_I, i_N_S_I, K_NH4, 
    # i_N_X_I, i_N_S_S, K_ALK, K_STO, b_A_NOX, i_N_X_S
    # b_H_NOX, b_STO_NOX, f_X_I, b_A_O2, i_N_BM, K_O2
# The following parameters are identifiable from the PLA graphs:
    # k_H, K_X, k_STO, K_S, eta_NOX, K_S, b_H_O2, b_STO_O2, mu_A, K_A_NH4, K_A_O2, Y_STO_O2, Y_STO_NOX, Y_H_O2, Y_A
# The others are non-identifiable: 
    # mu_H, K_A_ALK, Y_H_NOX,
# MAYBE Special cases (for those going to 0):
    # K_NOX



# FIXED - Specify which are fixed (get lilst of names from parameters to always fix)
model_fixed_params = list(params_to_always_fix.keys())
# PCA - Parameters (in order) for pca results
model_pca_params = pca_param_names.copy()
# SPECIAL case for 0 (1e-6) parameters -- TODO: MANUAL
model_zero_params = ['K_NOX']
# FREE - All other parameters are free to sample
model_free_params = [param for param in priors.keys() if param not in model_fixed_params and param not in model_pca_params and param not in model_zero_params]

# Print the parameters to be fixed, PCA, zero, and free
print(f'--------------------------------------------------------')
print(f'Parameters to be fixed (Param, value):')
print(f'--------------------------------------------------------')
for param in model_fixed_params:
    print(f'    {param}')
print(f'--------------------------------------------------------')
print(f'Parameters to be PCA transformed (Param):')
print(f'--------------------------------------------------------')
for param in model_pca_params:
    print(f'    {param}')
print(f'--------------------------------------------------------')
print(f'Parameters to be set to zero (Param):')
print(f'--------------------------------------------------------')
for param in model_zero_params:
    print(f'    {param}')
print(f'--------------------------------------------------------')
print(f'Parameters to be sampled (Param):')
print(f'--------------------------------------------------------')

for param in model_free_params:
    print(f'    {param}')
print(f'--------------------------------------------------------')
print(f'\nEnsure that the parameters to be fixed, PCA, zero, and free are correct before proceeding with the model')


In [ ]:
# ----------------------------------------------------------
## Residuals
# ----------------------------------------------------------

### Objective Function definition - Least squares
def obj_fun_lstsq(ode_function, theta, y0, t_eval, output_data, solver_method=solver_method, always_fixed_param_idxs=None, always_fixed_param_vals=None):
    """
    Objective function to compute residuals for least_squares optimization. Residuals are the differences between model predictions and true output data.

    Args:
        ode_function: function, the ODE system to solve, as f(t, y, theta)
        theta: array, parameters for the ODE system
        y0: array, initial values for the ODE system
        t_eval: array, time points to evaluate the ODE system at
        output_data: dataframe, true output data to compare against
        solver_method: str, method to use for solving the ODE system (default is 'BDF')
    Returns:
        residuals: array, the residuals (differences between model predictions and true output data)

    """
    # raw output data
    raw_time = output_data['Time'].values
    raw_output_data = output_data.drop(columns=['Time', 'Flowrate'])
    column_labels = raw_output_data.columns

    # Fix always fixed parameters if provided
    if always_fixed_param_idxs is not None:
        for idx, val in zip(always_fixed_param_idxs, always_fixed_param_vals):
            theta[idx] = val

    # Solve the ODE system with the given parameters
    ode_system = lambda t, y: ode_function(t, y, theta)
    t_span = (raw_time.min(), raw_time.max())
    sol = solve_ivp(fun=ode_system, t_span=t_span, y0=y0, t_eval=t_eval, method=solver_method, 
                    # rtol=1e-6, atol=1e-6
                    )
    model_time = sol.t
    # model_data = sol.y.T
    interp_func = interp1d(model_time, sol.y, kind='linear', axis=1, fill_value='extrapolate')

    # Interpolate model data to same time points as output_data
    model_data = interp_func(raw_time).T

    # Create a DataFrame for model data with the same columns as output_data
    model_interp_data = pd.DataFrame(model_data, columns=column_labels, index=raw_time)

    # Residuals calculation
    residuals = (raw_output_data - model_interp_data.values).values.flatten() # Flatten the residuals to a 1D array
    return residuals # Flatten the residuals to a 1D array for least_squares optimization
# End of obj_fun_lstsq() function

In [ ]:
# -----------------------------------------------------------
## Regression / Bootstrap Functions
# -----------------------------------------------------------

state_names = [
    'S_O2', 'S_I', 'S_S', 'S_NH4', 'S_N2', 
    'S_NOX', 'S_ALK', 'X_I', 'X_S', 'X_H', 
    'X_STO', 'X_A', 'X_SS'
]

def run_bootstrap_sample(
        sample_num,
        ode_function,
        theta_initial,
        y0,
        t_eval,
        output_data,
        solver_method=solver_method,
        always_fixed_param_idxs=always_fixed_param_idxs,
        always_fixed_param_vals=always_fixed_param_vals,
        bounds=None,
        ):
    """
    TODO
    """
    # Resample data with replacement - Same size as output_data with pandas - should have same columns
    resampled_data_length = int(len(output_data))
    resampled_data = output_data.sample(n=resampled_data_length, replace=True, axis=0)
    # Order resampled data according to time column
    resampled_data = resampled_data.sort_values(by='Time').reset_index(drop=True)

    # Set the parameter initial guess
    x0 = theta_initial.copy()

    # Call the original objective function with theta_model, that now contains the PCA transformed parameters
    residuals_fun = lambda x: obj_fun_lstsq(ode_function=ode_function, theta=x, y0=y0, t_eval=t_eval, output_data=resampled_data, solver_method=solver_method, 
                                            always_fixed_param_idxs=always_fixed_param_idxs, always_fixed_param_vals=always_fixed_param_vals)

    # Run optimization
    result = least_squares(
        fun=residuals_fun,
        x0=x0,
        bounds=bounds,
        # method='trf',
        # jac='3-point',
        # Set tolerances
        # xtol=1e-6, ftol=1e-6, gtol=1e-6
    )

    obj_val = np.sum(result.fun**2)
    theta_model = result.x

    # Print results for debugging
    # print(f'Sample {sample_num+1}: Objective value: {obj_val:.3e}, Success: {result.success}')

    return {
        'Sample': sample_num+1,
        'Parameter_outputs': {param: theta_model[all_param_idx[param]] for param in all_param_idx.keys()},
        'Residuals': result.fun,
        'Success': result.success,
        'Objective_value': obj_val,
    }
# End of run_bootstrap_sample() function

In [ ]:
# -----------------------------------------------------------
## Run Bootstrap Sampling
# -----------------------------------------------------------

## TODO - MANUAL
use_fixed_params = False  # Set to True to use fixed parameters, False to use all parameters
use_true_vals = True  # Set to True to use true values as initial guess, False to use midpoint of ranges
num_bootstrap_samples = 100
n_parallel_jobs = 10
# Overwrite the solver method to use for ODE solving
solver_method = 'BDF'  # 'LSODA', 'RK45', 'RK23', 'Radau', 'BDF', 'RKF45', 'RKF23', 'DOP853'

# If using fixed parameters, overwrite the always_fixed_param_idxs and always_fixed_param_vals with true values
if use_true_vals and use_fixed_params:
    for idx in always_fixed_param_idxs:
        always_fixed_param_vals[idx] = true_theta[all_param_idx[idx]]

# overwrite the always_fixed_param_idxs and always_fixed_param_vals if not using fixed parameters
if not use_fixed_params:
    always_fixed_param_idxs = None
    always_fixed_param_vals = None


##  Set initial guess for parameters
theta_initial = np.zeros(len(all_param_idx))
# For all parameters, set initial guess to the midpoint of the range
for i, param in enumerate(all_param_idx.keys()):
    theta_initial[i] = (theta_ranges[param][0] + theta_ranges[param][1]) / 2

# For fixed parameters, set initial guess to the fixed value
if use_fixed_params:
    for i, val in zip(always_fixed_param_idxs, always_fixed_param_vals):
        theta_initial[i] = val
if use_true_vals:
    for i, param in enumerate(all_param_idx.keys()):
        theta_initial[i] = true_theta[param]  

# --------

### ODE SYSTEM
# Time from data
bootstrap_t_eval = Data_Influent_states['Time'].values
# bootstrap_t_eval = np.linspace(Data_Influent_states['Time'].min(), Data_Influent_states['Time'].max(), num=1000)
# Initial values
y0 = get_reactor_initial_values(top_dir=top_dir)
# Interpolate influent data to match the time points of the bootstrap evaluation
interpolated_influent_data = Data_Influent_states.copy()
interpolated_influent_data = interpolated_influent_data.set_index('Time').reindex(bootstrap_t_eval).interpolate(method='linear').reset_index()
# Interpolated output data
interpolated_output_data = Data_Effluent_states.copy()
interpolated_output_data = interpolated_output_data.set_index('Time').reindex(bootstrap_t_eval).interpolate(method='linear').reset_index()

# Ode system
ode_system = lambda t, y, theta: ode_system_wrapper(t=t, y=y, theta=theta, influentData=interpolated_influent_data.to_numpy(), reactorVolumes=[r1_V])

## Bounds for parameters for least_squares optimization
bounds = []
# Normal parameters - set lower bound of 0 and upper bound of infinity
for _, _ in enumerate(all_param_idx):
    bounds.append((0, np.inf))

# Always fixed parameters - set bounds to a very small range around the fixed value
if use_fixed_params:
    for idx, val in zip(always_fixed_param_idxs, always_fixed_param_vals):
        bounds[idx] = (float(val - 1e-6), float(val + 1e-6))

# Convert to arrays of consistent type for least_squares
bounds_lstsq = (
    np.array([b[0] for b in bounds], dtype=np.float64),
    np.array([b[1] for b in bounds], dtype=np.float64)
)

# Initialize the resultsDataFrame
columns = ['Sample', 'Parameter_outputs', 'Residuals', 'Success', 'Objective_value']
index = range(1, num_bootstrap_samples + 1)
results_df = pd.DataFrame(index=index, columns=columns)

In [ ]:
## TODO: YOU CAN SKIP THIS IF YOU ALREADY HAVE BOOTSTRAP RESULTS PKL FILE

# -----------------------------------------------------------
## Run Bootstrap Sampling
# -----------------------------------------------------------

print(f"--------------------------------------------------------")
print(f"Bootstrap Sampling: \n  Are some parameters fixed? {use_fixed_params}\n Are true values used as initial guess? {use_true_vals}")
print(f"Running bootstrap sampling with {num_bootstrap_samples} samples in parallel, with {n_parallel_jobs} parallel jobs...\n")

# parallel version
results_list = Parallel(n_jobs=n_parallel_jobs)(
    delayed(run_bootstrap_sample)(
    sample_num=i,
    ode_function=ode_system,
    theta_initial=theta_initial,
    y0=y0,
    # t_eval=bootstrap_t_eval,
    t_eval = None, #TODO: testing purposes
    output_data=interpolated_output_data,
    solver_method=solver_method,
    always_fixed_param_idxs=always_fixed_param_idxs,
    always_fixed_param_vals=always_fixed_param_vals,
    bounds=bounds_lstsq
    )
    for i in tqdm(range(num_bootstrap_samples))
)

# non-parallel version
# results_list = []
# for i in tqdm(range(num_bootstrap_samples)):
#     res = run_bootstrap_sample(
#         sample_num=i,
#         ode_function=ode_system,
#         theta_initial=theta_initial,
#         y0=y0,
#         # t_eval=bootstrap_t_eval,
#         t_eval = None, #TODO: testing purposes
#         output_data=interpolated_output_data,
#         solver_method=solver_method,
#         always_fixed_param_idxs=always_fixed_param_idxs,
#         always_fixed_param_vals=always_fixed_param_vals,
#         bounds=bounds_lstsq
#     )
#     results_list.append(res)
#     print(f"Bootstrap Sample {i + 1}/{num_bootstrap_samples} completed. Success: {res['Success']}, Objective Value: {res['Objective_value']:.3e}")

# store results
for i, res in enumerate(results_list):
    results_df.at[i + 1, 'Sample'] = res['Sample']
    results_df.at[i + 1, 'Parameter_outputs'] = res['Parameter_outputs']
    results_df.at[i + 1, 'Residuals'] = res['Residuals']
    results_df.at[i + 1, 'Success'] = res['Success']
    results_df.at[i + 1, 'Objective_value'] = res['Objective_value']
    # print(f"Bootstrap Sample {i + 1}/{num_bootstrap_samples} completed. Success: {res['Success']}, Objective Value: {res['Objective_value']:.3e}")

# Save results to pickle file
results_df.to_pickle(results_dir / 'bootstrap_results.pkl')

In [ ]:
# -----------------------------------------------------------
## Output simulations of each bootstrap sample
# -----------------------------------------------------------

# Load results from pickle file
results_df = pd.read_pickle(results_dir / 'bootstrap_results.pkl')

# Extract parameter outputs, then convert to dataframe with parameter names as columns
param_df_columns = list(all_param_idx.keys())
parameter_outputs_df = pd.DataFrame(results_df['Parameter_outputs'].tolist(), index=results_df.index)
parameter_outputs_df.columns = param_df_columns

# Plot each bootstrap sample on output plots (COD, NH4, NOx, TKN, Alkalinity, TSS)

Data_Effluent_compounds_keys = ['COD', 'NH4+NH3', 'NO3+NO2', 'TKN', 'Alkalinity', 'TSS']
Data_Effluent_compounds_new_keys = ['COD', 'NH4', 'NOx', 'TKN', 'Alkalinity', 'TSS']
# map Data_Effluent_compounds to the expected keys
Data_Effluent_compounds_renamed = Data_Effluent_compounds.copy()
Data_Effluent_compounds_renamed = Data_Effluent_compounds_renamed.rename(columns={
    'NH4+NH3': 'NH4',
    'NO3+NO2': 'NOx'
})

bootstrap_t_eval = np.linspace(Data_Influent_states['Time'].min(), Data_Influent_states['Time'].max(), num=1000)
# For each parameter_output, simulate the model with the parameters and save in compound_bootstrap_data_df dataframe
t_span_bootstrap = (Data_Effluent_states['Time'].min(), Data_Effluent_states['Time'].max())
# Loop through each bootstrap sample and simulate the model

def simulate_bootstrap_sample(sample_idx, theta_current_sample):
    # ODE function for current sample
    ode_func_sample = lambda t, y: ode_system(t=t, y=y, theta=theta_current_sample)
    
    # Solve ODE
    sol_sample = solve_ivp(
        fun=ode_func_sample,
        t_span=t_span_bootstrap,
        y0=y0,
        t_eval=None,  # optional: can be bootstrap_t_eval
        method=solver_method
    )
    
    # Interpolate solution to bootstrap_t_eval points
    interp_func = interp1d(sol_sample.t, sol_sample.y, kind='linear', axis=1, fill_value='extrapolate')
    sample_state_vals = interp_func(bootstrap_t_eval).T
    
    # Build DataFrame of states
    sample_og_state_df = pd.DataFrame(data=sample_state_vals, columns=state_names, index=bootstrap_t_eval)
    sample_compound_result_df = pd.DataFrame(index=bootstrap_t_eval, columns=Data_Effluent_compounds_new_keys)
    
    # Derived compounds
    sample_compound_result_df['COD'] = sample_og_state_df[['S_I','S_S','X_I','X_S','X_H','X_A','X_STO']].sum(axis=1)
    sample_compound_result_df['NH4'] = sample_og_state_df['S_NH4']
    sample_compound_result_df['NOx'] = sample_og_state_df['S_NOX']
    sample_compound_result_df['TKN'] = sample_og_state_df['S_NH4'] + sample_og_state_df['S_N2']
    sample_compound_result_df['Alkalinity'] = sample_og_state_df['S_ALK']
    sample_compound_result_df['TSS'] = sample_og_state_df['X_SS']
    
    # Clip negative values
    sample_compound_result_df = sample_compound_result_df.clip(lower=0)
    
    print(f"Bootstrap Sample {sample_idx+1}/{len(parameter_outputs_df)} simulated.")
    return sample_compound_result_df.values

print(f"--------------------------------------------------------")
print(f"Simulating {len(parameter_outputs_df)} bootstrap samples in parallel...\n")

compound_bootstrap_data_parallel = Parallel(n_jobs=n_parallel_jobs)(
    delayed(simulate_bootstrap_sample)(i, row.values)
    for i, row in tqdm(parameter_outputs_df.iterrows())
)

# Convert to 3D numpy array
compound_bootstrap_data_array = np.array(compound_bootstrap_data_parallel)
n_samples, n_time_points, n_compounds = compound_bootstrap_data_array.shape

# Build MultiIndex DataFrame
multi_index = pd.MultiIndex.from_product(
    [range(n_samples), bootstrap_t_eval], names=['Sample', 'Time']
)
flat_data = compound_bootstrap_data_array.reshape(-1, n_compounds)

compound_bootstrap_data_df = pd.DataFrame(
    data=flat_data,
    index=multi_index,
    columns=Data_Effluent_compounds_new_keys
)

print('-' * 50)
print(f'compound_bootstrap_data_df shape: {compound_bootstrap_data_df.shape}')
print(compound_bootstrap_data_df.head())


## Underlying true values for compounds
sol_true = solve_ivp(
    fun=ode_system,
    t_span=t_span_bootstrap,
    y0=y0,
    t_eval=bootstrap_t_eval,
    method=solver_method,
    args=(true_theta_array,)
)
interp_func_true = interp1d(sol_true.t, sol_true.y, kind='linear', axis=1, fill_value='extrapolate')
true_state_vals = interp_func_true(bootstrap_t_eval).T 
true_og_state_df = pd.DataFrame(data=true_state_vals, columns=state_names, index=bootstrap_t_eval)
true_compound_result_df = pd.DataFrame(index=bootstrap_t_eval, columns=Data_Effluent_compounds_new_keys)
# Derived compounds
true_compound_result_df['COD'] = true_og_state_df[['S_I','S_S','X_I','X_S','X_H','X_A','X_STO']].sum(axis=1)
true_compound_result_df['NH4'] = true_og_state_df['S_NH4']
true_compound_result_df['NOx'] = true_og_state_df['S_NOX']
true_compound_result_df['TKN'] = true_og_state_df['S_NH4'] + true_og_state_df['S_N2']
true_compound_result_df['Alkalinity'] = true_og_state_df['S_ALK']
true_compound_result_df['TSS'] = true_og_state_df['X_SS']
# Clip negative values
true_compound_result_df = true_compound_result_df.clip(lower=0)


In [ ]:
# ------------------------------------------------------------
## Plot Bootstrap Lines
# ------------------------------------------------------------
fontsize = 24

# Function to plot all bootstrap lines for a compound
def plot_bootstrap_lines(compound_bootstrap_data_df, raw_effluent_data, output_col='COD'):
    plt.figure(figsize=(24, 10))

    # Plot raw data points
    plt.plot(raw_effluent_data['Time'], raw_effluent_data[output_col], 
             color='black', linewidth=0, marker='o', markersize=5, label='Raw Data')

    # Plot all bootstrap sample lines
    for sample_idx, sample_df in compound_bootstrap_data_df.groupby(level='Sample'):
        # Convert MultiIndex to columns
        sample_df_reset = sample_df.reset_index()
        plt.plot(sample_df_reset['Time'], sample_df_reset[output_col],
                 color='skyblue', alpha=0.4, linewidth=1)

    plt.xlabel('Time (d)', fontsize=fontsize)
    plt.ylabel(output_col, fontsize=fontsize)
    plt.title(f'Bootstrap Lines for {output_col}', fontsize=fontsize)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.grid(False)
    plt.tight_layout()
    plt.show()

# Function to plot bootstrap confidence intervals for a compound
def plot_bootstrap_confidence_intervals(compound_bootstrap_data_df, raw_effluent_data, output_col='COD', alpha=0.05):
    plt.figure(figsize=(24, 10))

    # Plot raw data points
    plt.plot(raw_effluent_data['Time'], raw_effluent_data[output_col], 
             color='black', linewidth=0, marker='o', markersize=5, label='Raw Data')

    # Calculate mean and confidence intervals
    mean_values = compound_bootstrap_data_df.groupby(level='Time')[output_col].mean()
    lower_bound = compound_bootstrap_data_df.groupby(level='Time')[output_col].quantile(alpha / 2)
    upper_bound = compound_bootstrap_data_df.groupby(level='Time')[output_col].quantile(1 - alpha / 2)

    # Plot mean line
    plt.plot(mean_values.index, mean_values.values, color='blue', linewidth=2, label='Mean')

    # Fill between for confidence intervals
    plt.fill_between(mean_values.index, lower_bound.values, upper_bound.values, 
                     color='lightblue', alpha=0.5, label=f'{100*(1-alpha):.0f}% CI')

    plt.xlabel('Time (d)', fontsize=fontsize)
    plt.ylabel(output_col, fontsize=fontsize)
    plt.title(f'Bootstrap Confidence Intervals for {output_col}', fontsize=fontsize)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.grid(False)
    plt.legend(fontsize=fontsize)
    plt.tight_layout()
    plt.show()

# Plot bootstrap lines and confidence regions for each compound
# COD
plot_bootstrap_lines(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='COD')
plot_bootstrap_confidence_intervals(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='COD')
# NH4
plot_bootstrap_lines(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='NH4')
plot_bootstrap_confidence_intervals(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='NH4')
# NOx
plot_bootstrap_lines(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='NOx')
plot_bootstrap_confidence_intervals(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='NOx')
# TKN
plot_bootstrap_lines(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='TKN')
plot_bootstrap_confidence_intervals(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='TKN')
# Alkalinity
plot_bootstrap_lines(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='Alkalinity')
plot_bootstrap_confidence_intervals(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='Alkalinity')
# TSS
plot_bootstrap_lines(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='TSS')
plot_bootstrap_confidence_intervals(compound_bootstrap_data_df, Data_Effluent_compounds_renamed, output_col='TSS')

                                    